## QHDOPT for quadratic programming

In this notebook, we demonstrate how to employ QHDOPT to solve a quadratic programming problem with box constraints. 

Our target problem is $$\min \ f(x)=\frac{1}{2}x^TQx+b^Tx,$$ where $Q = \begin{bmatrix}-2 & 1 \\ 1 & -1 \end{bmatrix}, b = \begin{bmatrix}\frac{3}{4} \\ -\frac{1}{4}\end{bmatrix},$ and $x$ is a 2-dimensional variable vector with each entry constrained in $[0, 1]$.

We employ the QP mode of QHDOPT to input this problem.

### 1. Create problem instance

First, we import the class QHD from our package QHDOPT, implying the solver's algorithm. 

In [ ]:
from qhdopt import QHD

Next, we construct the matrices $Q$ and $b$ by Python lists. For the matrix $Q$, it is represented by a nested list. The vector $b$ is represented by a list, encoding its transposed matrix $b^T$. 

In [ ]:
Q = [[-2, 1],
     [1, -1]]
bt = [3/4, -1/4]

Then we create a problem instance, stored in a variable `model`. It mandates the matrices $Q$ and $b$ to construct the problem, and by default set the box constraints to the unit box $[0, 1]^n$. You may override the bounds by `bounds=(l, r)` or `bounds=[(l1, r1), ..., (ln, rn)]`. 

In [ ]:
model = QHD.QP(Q, bt)

### 2. Solve with D-Wave

Now we illustrate how to solve the problem with QHDOPT's solvers. We consider the D-Wave solver first. 

We can configure the D-Wave solver by running `model.dwave_setup` with all the parameters set. The mandatory parameter is the resolution $r$, which we set as 8. The API key can be either directly input by setting `api_key` or from a file. 

You may also set the annealing schedule, chain strength, embedding schemes, etc. Here we use default parameters. 

In [ ]:
model.dwave_setup(resolution=8, api_key_from_file='/Users/samuelkushnir/Documents/dwave_api_key.txt')

To compile, send the job to run, and post-processing, you can run `model.optimize`. Setting `verbose=1` outputs more runtime information.

In [ ]:
response = model.optimize(verbose = 1)

Here, the coarse solution is one of the decoded solutions directly from D-Wave devices, fined-tuned solution is the best solution obtained by using classical local solvers to refine the coarse solutions, and the success rate is the portion of samples leading to the best solution.

The D-Wave solver returns a global minimum at $x=\begin{bmatrix} 0 \\ 1 \end{bmatrix}$, with the minimum value $-0.75$. After fine-tuning, the minimum does not change in this case. 

A runtime breakdown is also provided to exhibit the time consumption of each step in the solver.

The Response object holds all relevant solution information in a structured way. It also contains more debugging and time information.

### 2. Compare with classical solvers

QHD is a quantum-classical hybrid algorithm, and we can compare its performance with classical-only solvers. QHDOPT contains a baseline backend where a random sampling procedure is followed by the specified post-processing method. Developers can also use it to debug the programmed `model`. 

In [ ]:
response = model.classically_optimize(solver="IPOPT", verbose=1)

In this minimal example, the classical solver performs well in the success rate and run time. In harder cases, QHDOPT with D-Wave backends normally performs better. 

### 3. Solve with QuTiP

The QHD algorithm can be deployed to different backends, thanks to SimuQ's Hamiltonian-based compilation scheme. Here we demonstrate how we can use a QuTiP-based solver to implement QHD and solve the QP problem. 

The workflow follows the same style. We first setup the QuTiP solver, then solve with `optimize`. 

In [ ]:
model.qutip_setup(resolution=6, time_discretization=40)

In [ ]:
model.optimize(verbose = 1)

### 4. Solve with IonQ

We can also solve the QP problem with IonQ backends. Similarly, we first setup the IonQ solver, then solve with `optimize`. 

In [ ]:
model.ionq_setup(resolution=6, api_key_from_file='../ionq_API_key', time_discretization=10, shots = 1000, on_simulator=True)

In [ ]:
model.optimize(verbose = 1)

### 5. Obtain compilation details

For developers who need further details of the compilation procedure, we can print the intermediate parameters with the model. 

The D-Wave backend supports printing the hyper parameters and the final Hamiltonian. We may set `compile_only` for the `optimize` method to stop before sending the task to actual backends. 

In [ ]:
model.dwave_setup(resolution=2, api_key_from_file='/Users/samuelkushnir/Documents/dwave_api_key.txt')
model.compile_only()

For QuTiP backend, QHDOPT can print the Hamiltonian. Notice that SimuQ stores quantum systems as piece-wise constant Hamiltonians, here we set the `time_discretization` to a small number. 

In [ ]:
model.qutip_setup(resolution=2, time_discretization=3)
model.optimize(verbose=2, compile_only=True)

For IonQ backend, QHDOPT can print the Hamiltonian and the compiled circuit.

In [ ]:
model.ionq_setup(resolution=2, api_key_from_file='../ionq_API_key', time_discretization=3, shots = 1000, on_simulator=True)
model.optimize(verbose=2, compile_only=True)